In [1]:
import pandas as pd
import numpy as np
import plotly.io as pio
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import pandas_datareader.data as web
import datetime as dt

pio.templates.default = 'presentation'

In [2]:
company = "BTC-USD"
start = dt.date(2015, 1, 1)
end = dt.date.today()

data = web.DataReader(company, "yahoo", start, end)
data = data[['Close']].copy()

In [3]:
data

Close
Date                    
2015-01-01    314.248993
2015-01-02    315.032013
2015-01-03    281.082001
2015-01-04    264.195007
2015-01-05    274.473999
...                  ...
2022-04-06  43206.738281
2022-04-07  43503.847656
2022-04-08  42287.664062
2022-04-09  42782.136719
2022-04-10  42742.640625

[2657 rows x 1 columns]

In [4]:
ma_s = 50
ma_l = 200

In [5]:
data.Close.rolling(50).mean()

Date
2015-01-01             NaN
2015-01-02             NaN
2015-01-03             NaN
2015-01-04             NaN
2015-01-05             NaN
                  ...     
2022-04-06    41764.803750
2022-04-07    41755.643516
2022-04-08    41790.636563
2022-04-09    41845.659766
2022-04-10    41898.069453
Name: Close, Length: 2657, dtype: float64

In [6]:
data["MA_S"] = data.Close.rolling(ma_s).mean()

In [7]:
data["MA_L"] = data.Close.rolling(ma_l).mean()

In [8]:
data

Close          MA_S          MA_L
Date                                                
2015-01-01    314.248993           NaN           NaN
2015-01-02    315.032013           NaN           NaN
2015-01-03    281.082001           NaN           NaN
2015-01-04    264.195007           NaN           NaN
2015-01-05    274.473999           NaN           NaN
...                  ...           ...           ...
2022-04-06  43206.738281  41764.803750  48243.001230
2022-04-07  43503.847656  41755.643516  48224.219375
2022-04-08  42287.664062  41790.636563  48221.438691
2022-04-09  42782.136719  41845.659766  48231.880996
2022-04-10  42742.640625  41898.069453  48227.721660

[2657 rows x 3 columns]

In [9]:
fig = px.line(data, x=data.index, y=["Close", "MA_S", "MA_L"], title=f"{company} with MA_S = {ma_s} and MA_L = {ma_l}",
              labels={"value":"Share Prices $"})
fig.show()

In [10]:
data.dropna(inplace = True)

In [11]:
data

Close          MA_S          MA_L
Date                                                
2015-07-19    273.614014    252.227199    247.008470
2015-07-20    278.980988    253.203018    246.832130
2015-07-21    275.833008    254.261159    246.636135
2015-07-22    277.221985    255.289539    246.616835
2015-07-23    276.049011    256.293039    246.676105
...                  ...           ...           ...
2022-04-06  43206.738281  41764.803750  48243.001230
2022-04-07  43503.847656  41755.643516  48224.219375
2022-04-08  42287.664062  41790.636563  48221.438691
2022-04-09  42782.136719  41845.659766  48231.880996
2022-04-10  42742.640625  41898.069453  48227.721660

[2458 rows x 3 columns]

In [12]:
fig = px.line(data.loc["2016"], x=data.loc["2016"].index, y=["Close", "MA_S", "MA_L"], title=f"{company} with MA_S = {ma_s} and MA_L = {ma_l}",
              labels={"value":"Share Prices $"})
fig.show()

In [13]:
data["position"] = np.where(data["MA_S"] > data["MA_L"], 1, -1 )

In [14]:
data

Close          MA_S          MA_L  position
Date                                                          
2015-07-19    273.614014    252.227199    247.008470         1
2015-07-20    278.980988    253.203018    246.832130         1
2015-07-21    275.833008    254.261159    246.636135         1
2015-07-22    277.221985    255.289539    246.616835         1
2015-07-23    276.049011    256.293039    246.676105         1
...                  ...           ...           ...       ...
2022-04-06  43206.738281  41764.803750  48243.001230        -1
2022-04-07  43503.847656  41755.643516  48224.219375        -1
2022-04-08  42287.664062  41790.636563  48221.438691        -1
2022-04-09  42782.136719  41845.659766  48231.880996        -1
2022-04-10  42742.640625  41898.069453  48227.721660        -1

[2458 rows x 4 columns]

In [15]:
fig = make_subplots(specs=[[{"secondary_y": True}]])
fig.add_trace(
    go.Scatter(x=data.index, y=data.Close, name="Close Price", opacity=0.35),
    secondary_y=False,
)
fig.add_trace(
    go.Scatter(x=data.index, y=data["MA_S"], name="MA_S"),
    secondary_y=False,
)
fig.add_trace(
    go.Scatter(x=data.index, y=data["MA_L"], name="MA_L"),
    secondary_y=False,
)
fig.add_trace(
    go.Scatter(x=data.index, y=data['position'], name="Position"), 
    secondary_y=True
)
# Add figure title
fig.update_layout(
    title_text=f"{company} Buy and Sell Signals"
)
# Set x-axis title
fig.update_xaxes(title_text="Date")
# Set y-axes titles
fig.update_yaxes(title_text="Close Price", secondary_y=False)
fig.update_yaxes(title_text="Buy and Sell Position", secondary_y=True)
fig.show()

In [16]:
data["returns"] = np.log(data.Close.div(data.Close.shift(1)))

In [17]:
data["strategy"] = data.position.shift(1) * data["returns"]

In [18]:
data

Close          MA_S          MA_L  position   returns  \
Date                                                                       
2015-07-19    273.614014    252.227199    247.008470         1       NaN   
2015-07-20    278.980988    253.203018    246.832130         1  0.019425   
2015-07-21    275.833008    254.261159    246.636135         1 -0.011348   
2015-07-22    277.221985    255.289539    246.616835         1  0.005023   
2015-07-23    276.049011    256.293039    246.676105         1 -0.004240   
...                  ...           ...           ...       ...       ...   
2022-04-06  43206.738281  41764.803750  48243.001230        -1 -0.052946   
2022-04-07  43503.847656  41755.643516  48224.219375        -1  0.006853   
2022-04-08  42287.664062  41790.636563  48221.438691        -1 -0.028354   
2022-04-09  42782.136719  41845.659766  48231.880996        -1  0.011625   
2022-04-10  42742.640625  41898.069453  48227.721660        -1 -0.000924   

            strategy  
Date                  
2015-07-19       NaN  
2015-07-20  0.019425  
2015-07-21 -0.011348  
2015-07-22  0.005023  
2015-07-23 -0.004240  
...              ...  
2022-04-06  0.052946  
2022-04-07 -0.006853  
2022-04-08  0.028354  
2022-04-09 -0.011625  
2022-04-10  0.000924  

[2458 rows x 6 columns]

In [19]:
data.dropna(inplace = True)

In [20]:
data[["returns", "strategy"]].sum() # absolute performance

returns     5.051234
strategy    3.674673
dtype: float64

In [21]:
data[["returns", "strategy"]].sum().apply(np.exp) # absolute performance

returns     156.215100
strategy     39.435776
dtype: float64

In [22]:
data[["returns", "strategy"]].mean() * 252 # annualized return

returns     0.518075
strategy    0.376890
dtype: float64

In [23]:
data[["returns", "strategy"]].std() * np.sqrt(252) # annualized risk

returns     0.624378
strategy    0.624779
dtype: float64

In [24]:
data["creturns"] = data["returns"].cumsum().apply(np.exp)
data["cstrategy"] = data["strategy"].cumsum().apply(np.exp)

In [25]:
data

Close          MA_S          MA_L  position   returns  \
Date                                                                       
2015-07-20    278.980988    253.203018    246.832130         1  0.019425   
2015-07-21    275.833008    254.261159    246.636135         1 -0.011348   
2015-07-22    277.221985    255.289539    246.616835         1  0.005023   
2015-07-23    276.049011    256.293039    246.676105         1 -0.004240   
2015-07-24    288.278015    257.572119    246.745125         1  0.043347   
...                  ...           ...           ...       ...       ...   
2022-04-06  43206.738281  41764.803750  48243.001230        -1 -0.052946   
2022-04-07  43503.847656  41755.643516  48224.219375        -1  0.006853   
2022-04-08  42287.664062  41790.636563  48221.438691        -1 -0.028354   
2022-04-09  42782.136719  41845.659766  48231.880996        -1  0.011625   
2022-04-10  42742.640625  41898.069453  48227.721660        -1 -0.000924   

            strategy    creturns  cstrategy  
Date                                         
2015-07-20  0.019425    1.019615   1.019615  
2015-07-21 -0.011348    1.008110   1.008110  
2015-07-22  0.005023    1.013186   1.013186  
2015-07-23 -0.004240    1.008899   1.008899  
2015-07-24  0.043347    1.053594   1.053594  
...              ...         ...        ...  
2022-04-06  0.052946  157.911277  39.012184  
2022-04-07 -0.006853  158.997147  38.745750  
2022-04-08  0.028354  154.552260  39.860069  
2022-04-09 -0.011625  156.359450  39.399370  
2022-04-10  0.000924  156.215100  39.435776  

[2457 rows x 8 columns]

In [26]:
outperf = data.cstrategy.iloc[-1] - data.creturns.iloc[-1]
outperf

-116.77932406951962